In [1]:
import sys

sys.path.append('../src/d00_utils/')
from db_utils import dbReadWriteClean, dbReadWriteViews

In [2]:
io_clean = dbReadWriteClean()
df_dcm = io_clean.get_table('meta_lite')
df_dcm.head()

/home/ubuntu/anaconda3/envs/usal_echo/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)
/home/ubuntu/anaconda3/envs/usal_echo/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2903: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


,dirname,filename,tag1,tag2,value,tags
0,44919,a_40B5SAIG,18,6024,3,"('0018', '6024')"
1,44919,a_40B5SAIG,18,6026,3,"('0018', '6026')"
2,44919,a_40B5SAIG,18,602c,0.0754717,"('0018', '602c')"
3,44919,a_40B5SAIG,18,602e,0.0754717,"('0018', '602e')"
4,44919,a_40B5SAIG,28,0004,YBR_FULL_422,"('0028', '0004')"


In [5]:
print(df_dcm.shape)

(18831851, 6)


In [6]:
io_views = dbReadWriteViews()
df_inst_all = io_views.get_table('instances_w_labels')

# Dataframe edits to match with other table
df_inst = df_inst_all
df_inst.rename(columns={"instancefilename": "filename"}, inplace=True)
df_inst["filename"] = df_inst['filename'].str.rstrip()
df_inst["filename"] = df_inst['filename'].str.slice_replace(stop=0,repl='a_')
print(df_inst.shape)
df_inst.head()

(32554, 5)


,sopinstanceuid,filename,instanceidk,view,studyidk
0,1.2.840.113663.1500.1.365900332.3.3.20120703.9...,a_8Z0BWWL5,696108,plax,39531
1,1.2.840.113663.1500.1.365900332.3.4.20120703.9...,a_8Z0BWX0M,696109,plax,39531
2,1.2.840.113663.1500.1.365900332.3.7.20120703.9...,a_8Z0BWZGH,696112,a4c,39531
3,1.2.840.113663.1500.1.365900332.3.1.20120703.9...,a_8Z0BXKCB,696140,plax,39533
4,1.2.840.113663.1500.1.365900332.3.14.20120703....,a_8Z0BXYUW,696157,plax,39533


In [8]:
# # Check to see how many instancefilenames match up
fn_list = df_inst['filename'].tolist()
# fn_list = [s.rstrip() for s in fn_list]
# fn_list_a = ['a_' + s for s in fn_list]

fn_list_dcm = df_dcm['filename'].tolist()
print(len(set(fn_list) & set(fn_list_dcm)))

32470


In [9]:
# Check to see how many studies line up; note - not unique!
dir_list = df_dcm['dirname'].tolist()
study_list = df_inst['studyidk'].tolist()
print(len(set(dir_list) & set(study_list)))

12701


In [10]:
merge_df = df_inst.merge(df_dcm, on ='filename')
merge_df.drop(columns=['sopinstanceuid', 'dirname', 'tags'], inplace=True)
print(merge_df.shape)
merge_df.head()

(643837, 7)


,filename,instanceidk,view,studyidk,tag1,tag2,value
0,a_8Z0BWWL5,696108,plax,39531,2,0010,1.2.840.10008.1.2.5
1,a_8Z0BWWL5,696108,plax,39531,8,0012,20120703
2,a_8Z0BWWL5,696108,plax,39531,8,0016,1.2.840.10008.5.1.4.1.1.6.1
3,a_8Z0BWWL5,696108,plax,39531,8,002a,20120703090118
4,a_8Z0BWWL5,696108,plax,39531,8,1090,iE33


In [45]:
df = merge_df

df_num_frames_t = df.loc[(df['value'] != '1') & (df['tag2'] == '0008')] 
print(df_num_frames_t.shape)
df_num_frames_t.head()

(27580, 7)


,filename,instanceidk,view,studyidk,tag1,tag2,value
37,a_8Z0BWX0M,696109,plax,39531,28,0008,69
57,a_8Z0BWZGH,696112,a4c,39531,28,0008,23
76,a_8Z0BXKCB,696140,plax,39533,28,0008,52
95,a_8Z0BXYUW,696157,plax,39533,28,0008,40
136,a_8ZGVB44R,696422,plax,39552,28,0008,72


In [75]:
# Get only tags that we care about
df = merge_df
df['tag1'] = df['tag1'].astype(str) # consistency with tag2

# Region Spatial Format: (0018,6012) - no values entered throughout db
df_region_sfa = df.loc[(df['tag1'] == '18') & (df['tag2'] == '6012')] 
df_num_frames = df.loc[(df['tag1'] == '28') & (df['tag2'] == '0008')] 
df_ultra_color = df.loc[(df['tag1'] == '28') & (df['tag2'] == '0014')]

In [50]:
print(df_num_frames.shape)
# df_num_frames.head()

(27580, 7)


In [73]:
# Remove any instances with less than ten frames
# Number of Frames (0028,0008) should be >1
frame_nums_str = df_num_frames['value'].tolist()
frame_nums_int = list(map(int, frame_nums_str))
frame_nums_int = sorted(list(set(frame_nums_int)))

frame_nums_final = []
for elem in frame_nums_int:
    if elem >= 10:
        frame_nums_final.append(elem)
frame_nums_final = list(map(str, frame_nums_final))

df = df_num_frames
df_num_frames_filt = df[df['value'].isin(frame_nums_final)] 

(27580, 7)
(27563, 7)


In [76]:
##################################################
# Ultrasound Color (0028,0014) should be '0'
## WAITING ON: new meta_lite from WT
#print(df_ultra_color['value'].value_counts())
df_ultra_color_filt = df_ultra_color[df_ultra_color['value'] == '0']

0         30843
1          1627
(US) 0        1
Name: value, dtype: int64
(32471, 7)
(30843, 7)


In [77]:
##########################################
# Filter out tag values we are not interested in, e.g.
# Region SFA (0018,6012) should be 1

print(df_region_sfa.shape)
print(df_region_sfa['value'].value_counts())

df_region_sfa_filt = df_region_sfa[df_region_sfa['value'] == '1']
print(df_region_sfa_filt.shape)

(34957, 7)
1         32470
2          2484
3             1
(US) 2        1
(US) 1        1
Name: value, dtype: int64
(32470, 7)


In [79]:
# Get instances that passed each filtering step
inst_1 = df_region_sfa_filt['instanceidk'].tolist()
inst_2 = df_num_frames_filt['instanceidk'].tolist()
inst_3 = df_ultra_color_filt['instanceidk'].tolist()

# Get instances that passed all filtering steps
inst_final = list(set(inst_1) & set(inst_2) & set(inst_3))
print(len(inst_1))
print(len(inst_2))
print(len(inst_3))
print(len(inst_final))

32470
27563
30843
25971


In [84]:
# Now we have df_tags, which is all instances filtered by tag values
# Use this to keep desired rows in original instances_w_labels table
# good_instances = list(set(df_tags['instanceidk'].tolist()))
# print('Number of good instances: ' + str(len(good_instances)))

df = df_inst_all
df = df[df['instanceidk'].isin(inst_final)]
df['filename'] = df['filename'].str.lstrip('a_')

df.head()

/home/ubuntu/anaconda3/envs/usal_echo/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,sopinstanceuid,filename,instanceidk,view,studyidk
1,1.2.840.113663.1500.1.365900332.3.4.20120703.9...,8Z0BWX0M,696109,plax,39531
2,1.2.840.113663.1500.1.365900332.3.7.20120703.9...,8Z0BWZGH,696112,a4c,39531
3,1.2.840.113663.1500.1.365900332.3.1.20120703.9...,8Z0BXKCB,696140,plax,39533
4,1.2.840.113663.1500.1.365900332.3.14.20120703....,8Z0BXYUW,696157,plax,39533
6,1.2.840.113663.1500.1.365900332.3.8.20120704.9...,8ZGVB44R,696422,plax,39552


In [ ]:
#io_views.save_to_db(df, 'instances_w_labels')
# ^^^ Note this overwrites original instances_w_labels
# When done with nb, add this script at end of filter_views.py